In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import RMSprop
from keras.utils.vis_utils import plot_model
from keras.initializers import glorot_uniform
from keras.initializers import he_uniform
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
#Read in complete csv
data = pd.read_csv('Air2009-2019_Complete.csv')

In [3]:
#Data normalization, normalizes each features in the range of 0-1
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

In [4]:
#Data split in a 60/20/20 ratio
total_size = len(data)
train_size = int(total_size * 0.6)
val_size = int(total_size * 0.2)
test_size = total_size - train_size - val_size

train_df = data[:train_size]
val_df = data[train_size:train_size+val_size]
test_df = data[train_size+val_size:]
print(len(train_df))
print(len(val_df))
print(len(test_df))

57711
19237
19237


In [5]:
#Model Hyperpamater definition and assignment
#Additional model_name uses the parameter names for the tensorboard model name so progress can be inspected
n_steps = 24
n_features = 7
n_epochs = 100
n_batch = 128
n_of_layers = 1
n_of_neurons = '128'
activation = 'tanh'
lr = 0.001
optimizer = Adam(learning_rate=lr)
optimizer_name = 'adam'
final = 'True_Newest'

model_name = f"S={n_steps}L={n_of_layers}N={n_of_neurons}A={activation}O={optimizer_name}LR={lr}B={n_batch}F={final}"
tensorboard = TensorBoard(log_dir=f'logs/fit/{model_name}')

In [6]:
#Function creates sequences from the data based on the amount of n_steps
def lstm_sequence(data, n_steps):
    x, y = [], []
    for i in range(n_steps, len(data)):
        x.append(data[i-n_steps:i, :])
        y.append(data[i, 4]) #i,passenger column
    x, y = np.array(x), np.array(y)
    return x, y

In [7]:
#Data sequences creates for each train, test dataset for training, validation and testing
x_train, y_train = lstm_sequence(train_df, n_steps)
x_val, y_val = lstm_sequence(val_df, n_steps)
x_test, y_test = lstm_sequence(test_df, n_steps)

In [8]:
#LSTM Architecture
model = Sequential()
model.add(LSTM(128, activation=activation,
                    kernel_initializer=glorot_uniform(seed=42),
                    input_shape=(n_steps, n_features)))
model.add(Dropout(rate=0.2,seed=42))
model.add(Dense(units=1, activation='linear', 
                         kernel_initializer=glorot_uniform(seed=42)))

model.compile(optimizer=optimizer,loss='mean_squared_error',metrics=['mean_absolute_error'])
early_stop = EarlyStopping(monitor='val_loss', patience=2, verbose=1)

In [9]:
#The model is fit the training data, with the parameters for epochs, batch declared
#Validation data is used to cross-validate aswell as by the early_stop callback
#Tensorboard callback also used for later inspection
train_model = model.fit(x_train, y_train, 
           epochs=n_epochs, 
           batch_size=n_batch,
           validation_data=(x_val, y_val),
           callbacks=[early_stop,tensorboard],
           verbose=1)

Epoch 1/100
451/451 [==============================] - 7s 10ms/step - loss: 0.0128 - mean_absolute_error: 0.0850 - val_loss: 0.0080 - val_mean_absolute_error: 0.0671
Epoch 2/100
451/451 [==============================] - 4s 8ms/step - loss: 0.0058 - mean_absolute_error: 0.0583 - val_loss: 0.0063 - val_mean_absolute_error: 0.0572
Epoch 3/100
451/451 [==============================] - 5s 10ms/step - loss: 0.0049 - mean_absolute_error: 0.0529 - val_loss: 0.0054 - val_mean_absolute_error: 0.0548
Epoch 4/100
451/451 [==============================] - 4s 8ms/step - loss: 0.0046 - mean_absolute_error: 0.0501 - val_loss: 0.0053 - val_mean_absolute_error: 0.0543
Epoch 5/100
451/451 [==============================] - 4s 9ms/step - loss: 0.0043 - mean_absolute_error: 0.0484 - val_loss: 0.0050 - val_mean_absolute_error: 0.0516
Epoch 6/100
451/451 [==============================] - 4s 8ms/step - loss: 0.0040 - mean_absolute_error: 0.0465 - val_loss: 0.0047 - val_mean_absolute_error: 0.0502
Epoch 7/

In [10]:
#LSTM Evaluation using the model.evaluate() method on all 3 datasets
#The corresponding MSE and RMSE printed for each evaluation
train_score = model.evaluate(x_train, y_train, verbose=0)
val_score = model.evaluate(x_val, y_val, verbose=0)
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Train Score: {:.5f} MSE ({:.5f} RMSE)'.format(train_score[0], np.sqrt(train_score[0])))
print('Validation Score: {:.5f} MSE ({:.5f} RMSE)'.format(val_score[0], np.sqrt(val_score[0])))
print('Test Score: {:.5f} MSE ({:.5f} RMSE)'.format(test_score[0], np.sqrt(test_score[0])))

Train Score: 0.00268 MSE (0.05178 RMSE)
Validation Score: 0.00377 MSE (0.06141 RMSE)
Test Score: 0.00445 MSE (0.06667 RMSE)


In [11]:
#Prediction code
y_pred = model.predict(x_test)
#Some reshaping necessary, involving making "dummy" numpy arrays filled with 0s, 
#Then fitting data into the dummy arrays in order to avoid shape errors when making the predictions
y_test_dummy = np.zeros((y_test.shape[0], 7))
y_pred_dummy = np.zeros((y_pred.shape[0], 7))

In [12]:
#Using ravel to replace each empty value in index 4 (column 5) with the y_test and y_pred sets
y_test_dummy[:, 4] = y_test.ravel()
y_pred_dummy[:, 4] = y_pred.ravel()

In [13]:
#Fitting the remaining values from each feature in the x_test, this means the features up until index 4
#And all features after 4, the resulting y_test_dummy and y_pred_dummmy are ready for inverse transform
y_test_dummy[:, :4] = x_test[:, -1, :4]
y_pred_dummy[:, :4] = x_test[:, -1, :4]
y_test_dummy[:, 5:] = x_test[:, -1, 5:]
y_pred_dummy[:, 5:] = x_test[:, -1, 5:]

In [14]:
#Inverse transform just removes the normalization to return the values in index 4 (passengers) to the realistic scale in order to derive insight from the predictions
y_test_inv = np.round(scaler.inverse_transform(y_test_dummy)[:, 4])
y_pred_inv = np.round(scaler.inverse_transform(y_pred_dummy)[:, 4])

In [18]:
#Print an x amount of actual vs predicted values
index = 0
for i in range(48):
    print(f"Index: {index}, Actual: {y_test_inv[i]}, Predicted: {y_pred_inv[i]}")
    index+=1
index = 0

Index: 0, Actual: 1541.0, Predicted: 1310.0
Index: 1, Actual: 2097.0, Predicted: 1743.0
Index: 2, Actual: 2398.0, Predicted: 2273.0
Index: 3, Actual: 2540.0, Predicted: 2889.0
Index: 4, Actual: 4294.0, Predicted: 3405.0
Index: 5, Actual: 5761.0, Predicted: 4508.0
Index: 6, Actual: 5273.0, Predicted: 5883.0
Index: 7, Actual: 6506.0, Predicted: 6495.0
Index: 8, Actual: 6654.0, Predicted: 6960.0
Index: 9, Actual: 6876.0, Predicted: 7280.0
Index: 10, Actual: 7036.0, Predicted: 7388.0
Index: 11, Actual: 7919.0, Predicted: 7206.0
Index: 12, Actual: 7723.0, Predicted: 6866.0
Index: 13, Actual: 6387.0, Predicted: 6264.0
Index: 14, Actual: 5894.0, Predicted: 5286.0
Index: 15, Actual: 4796.0, Predicted: 4393.0
Index: 16, Actual: 2184.0, Predicted: 2331.0
Index: 17, Actual: 499.0, Predicted: 513.0
Index: 18, Actual: 181.0, Predicted: 115.0
Index: 19, Actual: 0.0, Predicted: 70.0
Index: 20, Actual: 0.0, Predicted: 79.0
Index: 21, Actual: 0.0, Predicted: 13.0
Index: 22, Actual: 99.0, Predicted: 11.

In [ ]:
#Tensorboard
#%load_ext tensorboard
#%tensorboard --logdir logs/fit